#Q6d.  Implement the LWR algorithm on the Housing Price data set with different tau values.	

In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
url = "https://raw.githubusercontent.com/ritesh1731/ML_ASSIGNMENT/main/Housingprice"
input_data = pd.read_csv(url)
input_data.head()

,Unnamed: 0,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,1,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,2,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,3,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,4,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,5,61000.0,6360,2,1,1,yes,no,no,no,no,0,no


In [73]:
FloorArea = input_data['lotsize']
Number_of_Bedrooms = input_data['bedrooms']
Number_of_Bathrooms = input_data['bathrms']
Price = input_data['price']
############################################

Y_Matrix = input_data['price']
Y_Matrix = np.array(Y_Matrix)
Y_Matrix = Y_Matrix.reshape(Y_Matrix.shape[0], 1)

#feature scaling
# Performing feature scanning on FloorArea
Mean_of_FloorArea = np.mean(FloorArea)
Max_of_FloorArea = max(FloorArea)
Min_of_FloorArea = min(FloorArea)
Scaled_FloorArea = []
for i in FloorArea:
	Scaled_FloorArea.append((i - Mean_of_FloorArea) / (Max_of_FloorArea - Min_of_FloorArea))


##Linear Weighted Regression 

In [82]:
def Error_Calculation(Y_Matrix, Y_pred):
    error = 0
    for i in range(len(Y_Matrix)):
        error += abs(Y_Matrix[i] - Y_pred[i]) / Y_Matrix[i]
    error /= len(Y_Matrix)
    return error * 100

def kernel(MatrixX, xi, Hyper_Parameter_Tau):
    return np.exp(-np.sum((xi - MatrixX) ** 2, axis = 1) / (2 * Hyper_Parameter_Tau * Hyper_Parameter_Tau))

def LocallyWeightedLR(MatrixX, xi, Y_Matrix, Hyper_Parameter_Tau):
	Transpose_X_Matrix = np.transpose(MatrixX)
	W = kernel(MatrixX, xi, Hyper_Parameter_Tau)
	MatrixXTransposeW = Transpose_X_Matrix * W
	MatrixXTransposeWX = np.matmul(MatrixXTransposeW, MatrixX)
	InverseMatrixXTransposeWX = np.linalg.pinv(MatrixXTransposeWX)
	InverseMatrixXTransposeWXMatrixXTransposeW = np.matmul(InverseMatrixXTransposeWX, MatrixXTransposeW)
	InverseMatrixXTransposeWXMatrixXTransposeWY = np.matmul(InverseMatrixXTransposeWXMatrixXTransposeW, Y_Matrix)
	InverseMatrixXTransposeWXMatrixXTransposeWYTranspose = np.transpose(InverseMatrixXTransposeWXMatrixXTransposeWY)
	return InverseMatrixXTransposeWXMatrixXTransposeWYTranspose.dot(xi)
 
MatrixX = []
for i in range(len(FloorArea)):
	MatrixX.append([1, Scaled_FloorArea[i], NoOfBedrooms[i], NoOfBathrooms[i]])
MatrixX = np.array(MatrixX)

# Hyper_Parameter_Tau = 0.00005
Taus = [0.1, 0.01, 0.02, 0.001, 0.0001, 0.00001]
for tau in Taus:
    print("Using Locally Weighted Linear Regression for Tau = " + str(tau))
    pred = []
    for i in range(MatrixX.shape[0]):
	    y_pred = LocallyWeightedLR(MatrixX, MatrixX[i], MatrixY, tau)
	    pred.append(y_pred)
    print("Mean absolute percentage error is : " + str(Error_Calculation(MatrixY,pred)))


Using Locally Weighted Linear Regression for Tau = 0.1
Mean absolute percentage error is : [20.14065581]
Using Locally Weighted Linear Regression for Tau = 0.01
Mean absolute percentage error is : [16.41583941]
Using Locally Weighted Linear Regression for Tau = 0.02
Mean absolute percentage error is : [18.12596846]
Using Locally Weighted Linear Regression for Tau = 0.001
Mean absolute percentage error is : [7.56066413]
Using Locally Weighted Linear Regression for Tau = 0.0001
Mean absolute percentage error is : [5.40856556]
Using Locally Weighted Linear Regression for Tau = 1e-05
Mean absolute percentage error is : [5.40730554]


##Batch grad_func descent with regularization 

In [83]:
############################################################
Area_of_FloorTrain = Scaled_Area_of_Floor[:383]
Number_of_BathroomsTrain = Number_of_Bathrooms[:383]
Number_of_BedroomsTrain = Number_of_Bedrooms[:383]
PriceTrain = Price[:383]
Area_of_FloorTest = Area_of_Floor[383:]
Number_of_BathroomsTest = Number_of_Bathrooms[383:]
Number_of_BedroomsTest = Number_of_Bedrooms[383:]
PriceTest = Price[383:]

#############################################################
X_Train = []
for i in range(383):
	X_Train.append([1, Scaled_Area_of_Floor[i], Number_of_Bedrooms[i], Number_of_Bathrooms[i]])
 
lr = 0.001  #learning rate 
lamda = -120  #lambda value

X_Test = []
n = len(Area_of_FloorTest)
for i in range(383,383+n):
  X_Test.append([1,Scaled_Area_of_Floor[i],Number_of_Bedrooms[i],Number_of_Bathrooms[i]])



x = np.array(X_Train)
y = np.array(PriceTrain)
w = np.array(([0],[0],[0],[0]))
y = y.reshape(len(y),1)
# print(w)

def multiply(w,x):
    ans=0
    for i in range(w.shape[0]):
        ans+=w[i][0]*x[i]
    return ans

def grad_func(x,w,y):
    ans=np.zeros(w.shape[0])
    for i in range(x.shape[0]):
        ans=(ans+(multiply(w,x[i])-y[i][0])*x[i])
    ans=ans.reshape(w.shape[0],1)
    return ans

m = x.shape[0]
for i in range(5000):
  temp = 1-((lr*lamda)/m)
  w = temp*w-lr*grad_func(x,w,y)/m

print("Cefficients are ",w)


# mean absolute percentage error
xtest = np.array(X_Test)
ytest = np.array(PriceTest)
ypred = np.dot(xtest,w)

n = xtest.shape[0]
mat = 0

for i in range(n):
  mat = mat + abs(ypred[i][0]-ytest[i])/y_test[i]
mat = mat*100/n
print("Mean absolute percentage error is ",mat)





Cefficients are  [[11027.93238871]
 [19272.80715199]
 [ 5262.5362372 ]
 [31018.09747139]]
Mean absolute percentage error is  18.84547313581711


##Stochastic grad_func descent with regulariation

In [80]:
X_Train = []
for i in range(383):
	X_Train.append([1, FloorArea_Scaled[i], NoOfBedrooms[i], NoOfBathrooms[i]])
 
X_Test = []
n = len(FloorAreaTest)
for i in range(383,383+n):
  X_Test.append([1,FloorArea_Scaled[i],NoOfBedrooms[i],NoOfBathrooms[i]])

lr = 0.0005
lamda = -10
x = np.array(X_Train)
y = np.array(PriceTrain)
w = np.array([0,0,0,0])
y = y.reshape(len(y),1)

print("initial coefficients : ",w)

def multiply(w,x):
    ans=0
    for i in range(x.shape[0]):
        ans= ans +w[i]*x[i]
    return ans


for itr in range(20):
  for i in range(x.shape[0]):
    grad = (multiply(w,x[i]) - y[i][0])*x[i]
    temp = 1-((lamda*lr)/x.shape[0])
    w = temp*w - lr*grad

print("coefficients are :",w)


# mean absolute percentage error
w = w.reshape((4,1))
xtest = np.array(X_Test)
ytest = np.array(PriceTest)
ypred = np.dot(xtest,w)

n = xtest.shape[0]
mat = 0

for i in range(n):
  mat = mat + abs(ypred[i][0]-ytest[i])/ytest[i]
mat = mat*100/n
print("Mean absolute percentage error",mat)

initial coefficients :  [0 0 0 0]
coefficients are : [ 8099.53322869  6266.87283801 13353.66990034 17698.12315756]
Mean absolute percentage error 18.655780791680915


##Mini Batch grad_func Descent with Regulatization 

In [77]:
X_Train = []
for i in range(383):
	X_Train.append([1, FloorArea_Scaled[i], NoOfBedrooms[i], NoOfBathrooms[i]])
 
X_Test = []
n = len(FloorAreaTest)
for i in range(383,383+n):
  X_Test.append([1,FloorArea_Scaled[i],NoOfBedrooms[i],NoOfBathrooms[i]])

bs = 20 
lr = 0.002
lamda = -20

x = np.array(X_Train)
y = np.array(PriceTrain)
w = np.array([0,0,0,0])
y = y.reshape(len(y),1)

def multiply(w,x):
    ans=0
    for i in range(x.shape[0]):
        ans= ans +w[i]*x[i]
    return ans

def grad_func(x,w,y,st,bs):
    ans=np.zeros(w.shape[0])
    for i in range(st,st+bs):
        if(i>=x.shape[0]):
          break
        ans=(ans+(multiply(w,x[i])-y[i][0])*x[i])
    return ans

for itr in range(50):
  for i in range(bs):
    temp = 1-((lamda*lr)/bs)
    w = temp*w - (lr*grad_func(x,w,y,bs*i,bs))/bs
print("coefficients are ",w)



# mean absolute percentage error
w = w.reshape((4,1))
xtest = np.array(X_Test)
ytest = np.array(PriceTest)
ypred = np.dot(xtest,w)

n = xtest.shape[0]
mat = 0

for i in range(n):
  mat = mat + abs(ypred[i][0]-ytest[i])/ytest[i]
mat = mat*100/n
print("Mean absolute percentage error : ",mat)

coefficients are  [ 9280.88414526 10001.13873579 10055.23366084 25309.01693085]
Mean absolute percentage error :  17.971898815464968
